# Notebook_4 - The SEIR Model

### Import needed packages

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# let's set things up so we have nice font sizes
plt.rcParams.update({'font.size': 12})

### Goals

We want to explore numerical solutions to the SEIR Model.

# An SEIR Model
I've implemented the SEIR equations in a simple (Euler) method. 

**Inputs**: 
- initial conditions (S0,E0,I0,R0) 
- params (alpha, beta, gamma)
- t a vector of evenly spaced times that you want to integrate over

**Outputs**:
- A tuple of (S,E,I,R) at the time points in t

In [ ]:
def SEIR(initial_conditions, params, t):
    '''
    This function forward integrates a basic SEIR model and returns S,E,I,R arrays.
    
    initial_conditions, a 4-tuple (S0, E0, I0, R0) with initial counts in compartments
    params, a 4-tuple (alpha,beta,gamma) with parameters:
        alpha, E->I symptom onset rate
        beta, infection rate per S-I contact
        gamma, I->R recovery rate
    t, an array of timepoints, ASSUMED TO BE EQUALLY SPACED. 
    
    Returns a numpy array of S, E, I, and R arrays. 
    '''
    S0, E0, I0, R0 = initial_conditions
    S, E, I, R = [S0], [E0], [I0], [R0]
    alpha, beta, gamma = params
    dt = t[1] - t[0]
    for _ in t[1:]:
        St = S[-1] - (beta*S[-1]*I[-1])*dt
        Et = E[-1] + (beta*S[-1]*I[-1] - alpha*E[-1])*dt
        It = I[-1] + (alpha*E[-1] - gamma*I[-1])*dt
        Rt = R[-1] + (gamma*I[-1])*dt
        S.append(St)
        E.append(Et)
        I.append(It)
        R.append(Rt)
    return S, E, I, R

In [ ]:
# Timesteps
N=1000
t_max = 150
dt = 0.1
t = np.linspace(0, t_max, int(t_max/dt) + 1)

# SEIR Dynamics
alpha = 0.2
beta = 1
gamma = 0.4
params = alpha, beta, gamma

In [ ]:
results = SEIR((1-1/N,0,1/N,0), params, t)

In [ ]:
# Set up the axes
fig, ax = plt.subplots(nrows=1,ncols=1)
S,E,I,R = results

# CODE TO PLOT GOES HERE.

# Make the plot attractive
ax.set_xlabel('')
ax.set_ylabel('')
ax.legend(loc='best')
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)

Use the code above to explore a few questions:

1. In the SIR model, we get an epidemic when $R_0=\frac{\beta}{\gamma}$ is $>1$. Does that hold here or not? Conduct some numerical experiments to see whether you think this $R_0$ holds. 

2. With all other parameters fixed, what effect does changing $\alpha$ have on the epidemic?  Explain this result in terms of what you know $\alpha$ does. 

3. We know that social distancing is meant to "flatten the curve". How could social distancing be incorporated into this model?

4. Suppose that hospital ICU beds will be totally taken up if more than 10% of the population is infectious at once. How much social distancing would be required if $\alpha = 0.2$, $\beta = 1$, and $\gamma=0.4$?